In [1]:
using Pkg
Pkg.activate("../STLCG.jl/.")

 Activating environment at `~/repos/AutonomousRiskFramework/STLCG.jl/Project.toml`


In [2]:
using Revise
using STLCG
using EllipsisNotation


┌ Info: Precompiling STLCG [6ababdf7-4701-4f28-b3b2-5630f64e3c98]
└ @ Base loading.jl:1278


## Short demo

In [4]:
# setting up the signals
bs = 1
t = 15
x_dim = 3
dim = 2
x = randn(bs, t, x_dim);
y = randn(bs, t, x_dim);


# rnn_cell(op, xs[1], op.h0; scale, distributed)
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

1×15×3 Array{Float64,3}:
[:, :, 1] =
 0.282462  -0.471594  1.00703  -0.680697  …  1.05104  0.0880835  -0.587167

[:, :, 2] =
 0.563332  0.534387  -0.661956  0.345958  …  0.585906  -1.42755  0.0722649

[:, :, 3] =
 -0.803788  -0.179061  -1.40908  …  1.6316  -0.81476  -2.06127  1.09988

In [21]:
formula = Then(subformula1=ϕ, subformula2=ψ, interval=[2, Inf])
trace = (x,y);
ρt(formula, trace)
vcat(STLCG.gradient(x -> sum(ρ(formula, x)), trace)[1]...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.0   0.0   1.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0

[:, :, 2] =
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -1.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   0.0   0.0   0.0   0.0

[:, :, 3] =
 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0  …  -0.0  -0.0  -0.0  -0.0  -0.0  -0.0
  0.0   0.0   0.0   0.0   0.0   0.0      0.0   0.0   1.0   0.0   0.0   0.0

### Make some formulas

In [7]:
θ = Negation(LessThan(:x, 4.0))
ϕ = LessThan(:x, 0.0)
ψ = GreaterThan(:y, 0.0)
Φ = Or(subformula1=ϕ, subformula2=ψ)
op = Always(subformula=Φ, interval=nothing)

Always
  subformula: Or
  interval: Nothing nothing
  _interval: Array{Float64}((2,)) [0.0, Inf]
  rnn_dim: Int8 1
  steps: Int8 1
  operation: Minish (function of type typeof(Minish))


In [8]:
# parameters
pscale=1    # scale for LessThan, GreaterThan, Equal
scale=0     # scale for the minish and maxish function used in temporal operators, implies, and, or
keepdims=true      # keep original dimension (should pretty much always be true)
distributed=false  # if there are multiple indices that have the same max/min values, then mean over those to the gradient flows through all those values

false

### Evaluating robustness trace

In [9]:
# there are two ways to do this
ϕ(x; pscale, scale, keepdims, distributed)
STLCG.robustness_trace(ϕ, x; pscale, scale, keepdims, distributed);

### Evaluating robustness $\rho$ (not trace)

In [10]:
# there are three ways to do this
# ρ(ϕ, x; pscale, scale, keepdims, distributed)
STLCG.robustness(ϕ, x; pscale, scale, keepdims, distributed)
ϕ(x; pscale, scale, keepdims, distributed)[:,end:end,..];


In [11]:
using Parameters # for @with_kw


In [7]:
@with_kw struct Test <: Formula
    subformula1
    subformula2
    operation = Maxish

    Test(ϕ, ψ) = new(ϕ, ψ, Maxish)
end

Test

In [15]:
@with_kw struct Poo1 <: TemporalFormula
    subformula
    interval
    _interval = (interval == nothing) ? [0, Inf] : interval
    rnn_dim = (interval == nothing) ? Int8(1) : (interval[end] == Inf ? Int8(interval[1]) : Int8(interval[2]))
    steps = (interval == nothing) ? Int8(1) : Int8(diff([1,5])[1] + 1)
    operation = Maxish
    
end

Poo1

In [19]:
Poo1(;subformula=ϕ, interval=nothing)

Poo1
  subformula: LessThan
  interval: Nothing nothing
  _interval: Array{Float64}((2,)) [0.0, Inf]
  rnn_dim: Int8 1
  steps: Int8 1
  operation: Maxish (function of type typeof(Maxish))


In [64]:
@with_kw struct Test2 <: TemporalFormula
    subformula1
    subformula2
    interval = nothing

    Test2(ϕ, ψ; interval=nothing) = new(ϕ, ϕ, interval) # example constructor
end

Test2

In [65]:
Test2(ϕ, ψ)

Test2
  subformula1: GreaterThan
  subformula2: GreaterThan
  interval: Nothing nothing


In [14]:
x = ones(1,20,1) .* collect(1:20)'
y = ones(1,20,1) .* collect(1:20)'
subformula1 = GreaterThan(:x, 0.0)
subformula2 = GreaterThan(:y, 5.0)
interval = [5,7]
N = 20


20

In [15]:
until = Until(;subformula1, subformula2, interval)
until((x,y))

1×20×1 Array{Float64,3}:
[:, :, 1] =
 -1.0e6  -1.0e6  -1.0e6  -1.0e6  …  4.0  5.0  6.0  7.0  8.0  9.0  10.0

In [35]:
LARGE_NUMBER = 1E6
a = 2
b = 4
duration = b - a +1
interval = [a,b]
trace1 = ϕ(x)
trace2 = ψ(y)
Alw = Always(subformula=GreaterThan(:z, 0.0), interval=nothing)
# LHS is [bs, t′, dim, time]
LHS = permutedims(repeat(reshape(trace2, (size(trace2)..., 1)), 1,1,1,size(trace2)[2]), [1, 4, 3, 2])
RHS = ones(size(LHS)) * -LARGE_NUMBER
for i in 1:size(trace2)[2]
    if (N-i+1-a) <= 0
        break
    end
    relevant = trace1[:,1:N-i+1]
    RHS[:,N-i+1,:,1:N-i+1-a] = Alw(relevant[:,end:-1:1,:]; pscale, scale, keepdims, distributed)[:,end:-1:a+1,:]
end
out = Maxish(Minish(cat(LHS, RHS, dims=5); dims=5, scale, keepdims=false, distributed); scale, keepdims=false, distributed, dims=4)

1×20×1 Array{Float64,3}:
[:, :, 1] =
 -1.0e6  -1.0e6  -4.0  -3.0  -2.0  -1.0  …  8.0  9.0  10.0  11.0  12.0  13.0

In [36]:
out[1,:,1]

20-element Array{Float64,1}:
 -1.0e6
 -1.0e6
 -4.0
 -3.0
 -2.0
 -1.0
  0.0
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0
 11.0
 12.0
 13.0

In [180]:
cat(trace1, trace2, dims=1)

2×20×1 Array{Float64,3}:
[:, :, 1] =
  1.0   2.0   3.0   4.0  5.0  6.0  7.0  …  15.0  16.0  17.0  18.0  19.0  20.0
 -4.0  -3.0  -2.0  -1.0  0.0  1.0  2.0     10.0  11.0  12.0  13.0  14.0  15.0

In [87]:
RHS = ones(size(LHS)) * -LARGE_NUMBER
for i in 1:size(trace2)[2]
    if (N-i+1-b) < 0
        break
    end

    RHS[:,N-i+1,:,N-i+1-b:N-i+1-a] = Alw(trace1[:,end-i+1:-1:1,:]; pscale, scale, keepdims, distributed)[:,end-duration+1:end,:]
end

LoadError: BoundsError: attempt to access 1×20×1×20 Array{Float64,4} at index [1:1, 4, 1:1, 0:2]

In [67]:
Alw(trace1[:,end:-1:i,:])[:,end-duration+1:end,:]

1×6×1 Array{Float64,3}:
[:, :, 1] =
 7.0  6.0  5.0  4.0  3.0  2.0

In [29]:
i = 1
(N-i+1-b) |> show
test = trace1

16

1×20×1 Array{Float64,3}:
[:, :, 1] =
 1.0  2.0  3.0  4.0  5.0  6.0  7.0  …  15.0  16.0  17.0  18.0  19.0  20.0

In [31]:
relevant = test[:,1:N-i+1]

1×20 Array{Float64,2}:
 1.0  2.0  3.0  4.0  5.0  6.0  7.0  …  15.0  16.0  17.0  18.0  19.0  20.0

In [34]:
Alw(relevant[:,end:-1:1,:]; pscale, scale, keepdims, distributed)[:,end:-1:a+1,:]

1×18×1 Array{Float64,3}:
[:, :, 1] =
 1.0  2.0  3.0  4.0  5.0  6.0  7.0  …  13.0  14.0  15.0  16.0  17.0  18.0

In [158]:

RHS[:,N-i+1,:,N-i+1-b:N-i+1-a] = 

1×3×1 Array{Float64,3}:
[:, :, 1] =
 16.0  17.0  18.0

In [161]:
RHS = ones(size(LHS)) * -LARGE_NUMBER
for i in 1:size(trace2)[2]
    if (N-i+1-b) <= 0
        break
    end
    relevant = trace1[:,N-i+1-b:N-i+1]
    RHS[:,N-i+1,:,N-i+1-b:N-i+1-a] = Alw(relevant[:,end:-1:1,:]; pscale, scale, keepdims, distributed)[:,end:-1:end-duration+1,:]
end

In [165]:
RHS[:,N-i+1,:,N-i+1-b:N-i+1-a]

1×1×3 Array{Float64,3}:
[:, :, 1] =
 16.0

[:, :, 2] =
 17.0

[:, :, 3] =
 18.0

In [170]:
trace1

1×20×1 Array{Float64,3}:
[:, :, 1] =
 1.0  2.0  3.0  4.0  5.0  6.0  7.0  …  15.0  16.0  17.0  18.0  19.0  20.0

In [174]:
RHS[1,6,1,:]

20-element Array{Float64,1}:
 -1.0e6
  2.0
  3.0
  4.0
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6
 -1.0e6

In [37]:
i = 2
duration = 6
RHS[:,end,1,i:i+duration]

1×7 Array{Float64,2}:
 2.0  3.0  4.0  5.0  6.0  7.0  8.0

In [32]:
RHS[:,:,1,i]

1×20 Array{Float64,2}:
 -1.0e6  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0

In [58]:
LHS = permutedims(repeat(reshape(trace2, (size(trace2)..., 1)), 1,1,1,size(trace2)[2]), [1, 4, 3, 2])
RHS = ones(size(LHS)) * -LARGE_NUMBER
for i in 1:size(trace2)[2]
    LHS[:,1:i-1,:,i]
    RHS[:,i:end,:,i] = Alw(trace1[:,i:end,:]; pscale, scale, keepdims, distributed)
end

15-element Array{Float64,1}:
 -1.0e6
 -1.0e6
 -1.0e6
 -1.1492900114485687
 -1.1492900114485687
 -1.212407114238258
 -1.212407114238258
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606
 -0.631478841925606

In [53]:
Maxish(Minish(cat(LHS, RHS, dims=5); dims=5, scale, keepdims=false, distributed); scale, keepdims=false, distributed, dims=4)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.110268  -0.570177  -0.570177  …   0.0423217   0.0423217  -0.177004
 0.607287   0.607287  -1.42709      -0.688285   -1.78225    -0.21453

[:, :, 2] =
  0.178606  -0.719736  -0.578093  …  -0.718358  -0.608101  -0.499044
 -0.853146  -0.853146   0.735788      0.538016   1.58135   -1.01577

[:, :, 3] =
  0.206686   0.206686  -1.53311   0.12955  …   0.331363   0.331363  -0.475145
 -0.348084  -0.802872  -1.35152  -1.20997     -0.526954  -0.792873   0.694338

In [96]:
vcat(STLCG.gradient(x -> sum(op((x,x); distributed=true)), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0

In [99]:
vcat(STLCG.gradient((x, y) -> sum(STLCG.robustness_trace(op, (x,y); distributed=true)), x,y)...)[:,:,1]'

15×4 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  1.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  1.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0

In [9]:
vcat(STLCG.gradient(x -> sum(STLCG.run_rnn_cell(op, x, x, scale=0)[1][:,end,:]), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [103]:
□ϕ = STLCG.□(op, interval=[2,4])

Always
  subformula: Always
  interval: Array{Int64}((2,)) [2, 4]
  _interval: Array{Int64}((2,)) [2, 4]
  rnn_dim: Int8 4
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))
  M: Array{Float64}((4, 4)) [0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0]
  b: Array{Float64}((4,)) [0.0, 0.0, 0.0, 1.0]


In [141]:
robustness_trace(□ϕ, x; args...)

LoadError: MethodError: no method matching robustness_trace(::Always, ::Array{Float64,3}; keepdims=true, dims=4, distributed=false, scale=0, pscale=1)
Closest candidates are:
  robustness_trace(!Matched::STLCG.Or, ::Any; dims, pscale, scale, keepdims, distributed) at In[54]:44
  robustness_trace(!Matched::STLCG.LessThan, ::Any; pscale, kwargs...) at In[54]:2
  robustness_trace(!Matched::STLCG.Implies, ::Any; pscale, scale, keepdims, distributed) at In[54]:7 got unsupported keyword argument "dims"
  ...

In [ ]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

In [4]:
ϕ = STLCG.LessThan(:x, 5)
ψ = STLCG.GreaterThan(:x, -3)
op = STLCG.Or(subformula1=STLCG.And(subformula1=ϕ, subformula2=ψ), subformula2=ϕ)

STLCG.Or
  subformula1: STLCG.And
  subformula2: STLCG.LessThan
  operation: Maxish (function of type typeof(Maxish))


In [191]:
(f::STLCG.Formula)(x::Array{Float64}; kwargs...) = robustness_trace(f, x; kwargs...)

In [192]:
ϕ(x)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 4.33878  4.55352  4.0096   4.28465  …  4.87325  4.24155  4.18938  4.36833
 4.35477  4.20984  4.53724  4.8834      4.25997  4.16101  4.62856  4.13871

[:, :, 2] =
 4.19292  4.05989  4.26052  4.3984   …  4.02068  4.74747  4.05824  4.06667
 4.13728  4.53258  4.77843  4.72391     4.18943  4.02867  4.20342  4.66529

[:, :, 3] =
 4.47263  4.89431  4.49562  4.28735  …  4.9606   4.3826   4.59063  4.70413
 4.29403  4.08497  4.01783  4.86847     4.10466  4.29218  4.78882  4.30469

In [147]:
struct Predicate
    x::Symbol
end

In [157]:
(p::Primitive)(x::Int) = (2*x, p.x)
(p::Primitive)(x::Real) = 3*x

In [158]:
a = Primitive(:asadsadsa)

Primitive(:asadsadsa)

In [159]:
a(1), a(1.)

((2, :asadsadsa), 3.0)

In [148]:
Primitive(:x) < 5

Primitive(:x)

In [172]:
Primitive(:a) > Primitive(:b)

false

In [170]:
(Base.:(<))(a::Symbol, b::Primitive) = a.x < b.x

In [75]:
function <(a,b)
    a < b
end

< (generic function with 1 method)

In [78]:
< |> methods

LoadError: syntax: "<" is not a unary operator